**input_vars**
8 times: 

* selected bit_rate (normalized by the max bit_rate), 
* buffer_size (in sec, normalized by 10), 
* bandwidth_measurement:throughput (kilobyte/ms) 
* bandwidth_measurement:time (in 10 sec), 
* next_chunk_size (in mega byte), 
* chunk_til_video_end (normalized by CHUNK_TIL_VIDEO_END_CAP)


VIDEO_BIT_RATE = [300,750,1200,1850,2850,4300]

CHUNK_TIL_VIDEO_END_CAP = 48.0

https://github.com/hongzimao/pensieve/blob/master/sim/rl_test.py

# Proporty Definition

## Robustness

In [6]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"
unused_input_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 38, 39, 40, 41, 42, 43, 44, 45, 46]


def get_percentage_input_range(range_arr, percentage):
    res = []
    for range in range_arr:
        span = range[1] - range[0]
        reduction = span * (1 - percentage / 100) / 2
        res.append((range[0] + reduction, range[1] - reduction))
    return res


def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, input_percentage, input_range):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        if i in unused_input_indices:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", 0))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range[int(i/8)][0]))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range[int(i/8)][1]))

    for i in range(model_input_size):
        if i % 8 == 7 and i not in unused_input_indices:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", -epsilon))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))
    
    path = base_path.format(input_percentage, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate)

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()

In [ ]:
model_input_size = 48
model_output_size = 6

epsilon = 0.01
tolerance = 0.40

base_path = "vnnlib/robustness/input{}_eps{}_a{}_b{}.vnnlib"
total_input_range = [(0.07, 1.0), (0.4, 1.4), (0.1, 1.1), (0.05, 0.5), (0.1, 0.5), (0.2, 1)]

for input_percentage in [60, 70, 80, 90, 100]:
    input_range = get_percentage_input_range(total_input_range, input_percentage)
    for i in range(model_output_size):
        for j in range(model_output_size):
            if (i - j) > tolerance * model_output_size or (j-i) > tolerance * model_output_size:
                gen_prop(copy_one_argmax_candidate=i, copy_two_argmax_candidate=j, input_percentage=input_percentage, input_range=input_range)


## Monotonicity

In [10]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"
unused_input_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 38, 39, 40, 41, 42, 43, 44, 45, 46]

def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, input_percentage, input_range, feat_index):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        if i in unused_input_indices:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", 0))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range[int(i/8)][0]))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range[int(i/8)][1]))

    for i in range(model_input_size):
        if i % 8 == 7 and int(i/8) in feat_index:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))

    path = base_path.format(input_percentage, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate)

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()

In [12]:
model_input_size = 48
model_output_size = 6

epsilon = 0.01
tolerance = 0.40
throughput_idx = [2]
buffer_idx = [1]
total_input_range = [(0.07, 1.0), (0.4, 1.4), (0.1, 1.1), (0.05, 0.5), (0.1, 0.5), (0.2, 1)]

# base_path = "vnnlib/capacity_utilization/input{}_eps{}_a{}_b{}.vnnlib"
base_path = "vnnlib/rebuffering_avoidance/input{}_eps{}_a{}_b{}.vnnlib"
for input_percentage in [60, 70, 80, 90, 100]:
    input_range = get_percentage_input_range(total_input_range, input_percentage)
    for i in range(model_output_size):
        for j in range(model_output_size):
            if (i-j) > tolerance * model_output_size:
                gen_prop(copy_one_argmax_candidate=i, copy_two_argmax_candidate=j, 
                         input_percentage=input_percentage, input_range=input_range,
                         feat_index=buffer_idx)


## Model Processing

#### Pensieve

In [7]:
from onnx2pytorch import ConvertModel
import onnx 
import torch 
import torch.nn as nn

def get_model(size="small"):
    assert size in ["small", "mid", "big"]
    
    path_to_onnx_model = f"/home/mzi/sys-rl-verif/VNNComp23_NN4Sys/onnx/pensieve_{size}_simple.onnx"
    onnx_model = onnx.load(path_to_onnx_model)
    pytorch_model = ConvertModel(onnx_model)

    return pytorch_model


#### Comparative

In [8]:
def get_params_argmax(input_size):
    
    # Take sum of the input vars
    c01 = torch.zeros([1, 1, input_size+1])
    c01[0][0][0] = 1

    c02 = torch.zeros([1, 1, input_size+1])
    c02[0][0][0] = 1
    c02[0][0][-1] = 1

    return c01, c02

def pensieve() -> nn.Sequential:
    base_model = get_model()

    class MyModel(nn.ModuleList):
        def __init__(self, device=torch.device("cpu")):
            super(MyModel, self).__init__()

            input_size = 48
            self.input_size = input_size
            c01, c02 = get_params_argmax(input_size)
            
            self.ft = torch.nn.Flatten()

            #################
            # Model
            ################# 
            self.base_model = base_model
            
            #################
            # Input summation
            #################
            self.input_conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=input_size+1)
            self.input_conv1.weight = torch.nn.Parameter(c01, requires_grad=True)
            self.input_conv1.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv1.bias, requires_grad=True))
            
            self.input_conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=input_size+1)
            self.input_conv2.weight = torch.nn.Parameter(c02, requires_grad=True)
            self.input_conv2.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv2.bias, requires_grad=True))            

        def forward(self, obs):
            # input processing
            input1 = self.input_conv1(obs)
            input2 = self.input_conv2(obs)
            
            # the model
            copy1_logits = self.base_model(input1)
            copy2_logits = self.base_model(input2)
            
            return self.ft(torch.concat((copy1_logits, copy2_logits), dim=1))

    model = MyModel()

    return model


## Input splitting

In [1]:
from itertools import product
import random

def split_range(r):
    lo, hi = r
    mid = (lo + hi) / 2
    return (lo, mid), (mid, hi)


def generate_split_lists(base_ranges, k, seed=None):
    """
    base_ranges: list of tuples, length 48
    k: number of variables to randomly choose and split
    seed: optional, for reproducibility
    """
    n = len(base_ranges)
    assert k <= n, "k cannot be larger than number of variables"

    rng = random.Random(seed)
    chosen_indices = rng.sample(range(n), k)
    chosen_set = set(chosen_indices)

    # For each position, build the list of options
    # - if not chosen: [original]
    # - if chosen: [lower_half, upper_half]
    options_per_position = []
    for i, r in enumerate(base_ranges):
        if i in chosen_set:
            low_half, high_half = split_range(r)
            options_per_position.append([low_half, high_half])
        else:
            options_per_position.append([r])

    # product(*options_per_position) yields tuples of length n,
    # each being one full configuration
    for combo in product(*options_per_position):
        yield list(combo)


input_range = [
    (0.07, 1.0), (0.07, 1.0), (0.07, 1.0), (0.07, 1.0),
    (0.07, 1.0), (0.07, 1.0), (0.07, 1.0), (0.07, 1.0),
    (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4),
    (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4),
    (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1),
    (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1),
    (0.05, 0.5), (0.05, 0.5), (0.05, 0.5), (0.05, 0.5),
    (0.05, 0.5), (0.05, 0.5), (0.05, 0.5), (0.05, 0.5),
    (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5),
    (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5),
    (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1),
    (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1),
]

k = 4  # for example

all_lists = list(generate_split_lists(input_range, k, seed=0))
print(all_lists[0])    # one example configuration


[(0.07, 1.0), (0.07, 1.0), (0.07, 0.535), (0.07, 1.0), (0.07, 1.0), (0.07, 1.0), (0.07, 1.0), (0.07, 1.0), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.4, 1.4), (0.1, 0.6000000000000001), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.1, 1.1), (0.05, 0.275), (0.05, 0.5), (0.05, 0.275), (0.05, 0.5), (0.05, 0.5), (0.05, 0.5), (0.05, 0.5), (0.05, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.1, 0.5), (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1), (0.2, 1)]


### Capcaity Util

In [2]:
DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"


def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, feat_index, idx, difficulty):
    input_range = all_lists[idx]
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range[i][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range[i][1]))

    for i in range(model_input_size):
        if i % 8 == 0 and int(i/6) in feat_index:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))
        else:
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0))
            lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", 0))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))

    path = base_path.format(difficulty, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate, idx)

    f = open(path, "w")
    f.write('\n'.join(lines) )
    f.close()

In [4]:
model_input_size = 48
model_output_size = 6
epsilon = 0.001
tolerance = 0.40
throughput_idx = [2]
buffer_idx = [1]
k = 4
all_lists = list(generate_split_lists(input_range, k, seed=0))

base_path = "vnnlib/capacity_utilization/splitted_input/difficulty{}_eps{}_a{}_b{}_k4_seed0_idx{}.vnnlib"
# base_path = "vnnlib/rebuffering_avoidance/difficulty{}_eps{}_a{}_b{}.vnnlib"
for i in range(model_output_size):
    for j in range(model_output_size):
        if (i-j) > tolerance * model_output_size:
            for idx in range(2 ** k):
                gen_prop(copy_one_argmax_candidate=i, copy_two_argmax_candidate=j, feat_index=throughput_idx, idx=idx, difficulty=3)
